<a href="https://colab.research.google.com/github/choarauc/form_ch/blob/main/SQLAVANCED_CORRECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 The top 3 most popular movies

In [ ]:
 SELECT m.original_title, m.popularity 
 FROM movies as m
 ORDER BY m.popularity DESC
 LIMIT 3;

# View

créer une VIEW qui contient les réalisatrices, chacun de leur film ainsi que le nombre de votant par film.
Enfin, à partir de cette vue, on veut le classement des réalisatrices en fonction du nombre de films, pour les films ayant plus de 1000 votants.


In [ ]:
#crée une vue nommée "realisatrice". 
# Les vues sont des objets de base de données qui stockent des requêtes SQL et renvoient des résultats dynamiques en fonction des données de la base de données.

CREATE VIEW realisatrice AS

# sélectionne les noms des réalisatrice et le nombre de votes que chaque film a obtenu
SELECT d.name, m.original_title, m.vote_count 

# que les données à récupérer proviennent de deux tables différentes, "directors" et "movies", qui sont jointes en utilisant la clé étrangère "id" de la table "directors" et la clé primaire "director_id" de la table "movies".
FROM directors d
JOIN movies m ON d.id = m.director_id

# filtre les réalisatrices en fonction de leur genre (le chiffre 1 étant souvent utilisé pour représenter les femmes dans les bases de données).
WHERE d.gender = 1;

# sélectionne les noms des réalisatrices et compte le nombre de films qu'elles ont réalisés, en utilisant la vue "realisatrice" créée à la ligne 1.
SELECT r.name, count(*) number_of_movies
FROM realisatrice r

# ligne filtre les réalisatrices en fonction du nombre de votes que leurs films ont reçus (en utilisant la colonne "vote_count" de la vue "realisatrice").
WHERE r.vote_count > 1000

# regroupe les données par nom de réalisatrice, afin que le nombre de films qu'elles ont réalisés puisse être compté.
GROUP BY r.name
ORDER BY number_of_movies DESC;


# Window Function 

Pour chaque réalisateur, Le top 3 des films dont la note moyenne est supérieure à 8 et réalisé après 2000, en fonction de la note moyenne.

In [ ]:
RANK()      
# sélectionne toutes les colonnes de la requête.
      SELECT *

# crée une sous-requête qui sélectionne les noms des réalisateurs, les titres des films qu'ils ont réalisés, la note moyenne des films et leur classement en fonction de leur note moyenne. 
           FROM ( SELECT d.name,
               m.original_title,
               m.vote_average,
               RANK() OVER( PARTITION BY d.name
                                       ORDER BY m.vote_average DESC ) as Ranking # La fonction RANK() OVER() permet de classer les films en fonction de leur note moyenne, en partitionnant les résultats par nom de réalisateur.
                    FROM movies m
                    INNER JOIN directors d ON m.director_id = d.id
                    WHERE m.vote_average > 8 AND m.release_date > 2000 ) # Les résultats sont filtrés pour n'inclure que les films ayant une note moyenne supérieure à 8 et une date de sortie ultérieure à l'an 2000
      WHERE Ranking < 4

## filtre les résultats pour n'afficher que les trois premiers films de chaque réalisateur, en utilisant le classement créé dans la sous-requête. 
# Les réalisateurs dont les films n'ont pas été classés dans les trois premiers ne seront pas inclus dans les résultats.

# Buckets


Pour chaque film, on aimerait une information supplémentaire (donc une colonne supplémentaire). Tu vas créer plusieurs intervalles:
[1$ - 100$, 100$ - 10k$, 10k$ - 100k$, 100k$ - 1M$, 1M$ - 100M$, 100M$ - 250M$, > 250M$]
En plus de cette information, tu afficheras pour chaque film: le nom du réalisateur, le titre du film et le budget.

In [ ]:

# sélectionne les noms des réalisateurs, les titres originaux des films et les budgets de chaque film
SELECT d.name, m.original_title, m.budget,

# L'opération "CASE" est utilisée pour établir des conditions et affecter une plage de budget spécifique à chaque film.
CASE WHEN m.budget BETWEEN 1 AND 100 THEN '1$ - 100$'
	 WHEN m.budget BETWEEN 100 AND 10000 THEN '100$ - 10k$'
	 WHEN m.budget BETWEEN 10000 AND 100000 THEN '10k$ - 100k$'
	 WHEN m.budget BETWEEN 100000 AND 1000000 THEN '100k$ - 1M$'
	 WHEN m.budget BETWEEN 1000000 AND 100000000 THEN '1M$ - 100M$'
	 WHEN m.budget BETWEEN 100000000 AND 250000000 THEN '100M$ - 250M$'
	 WHEN m.budget > 250000000 THEN '> 250M$'
	 ELSE '0'
	 END AS Buckets_budget_$ #  ajoute une colonne supplémentaire "Buckets_budget_$" en fonction des plages de budgets auxquelles chaque film appartient.


FROM movies m
JOIN directors d ON m.director_id = d.id
ORDER BY m.budget DESC; #résultats par ordre décroissant de budget, de sorte que les films avec le budget le plus élevé apparaissent en premier dans les résultats.

# MAX() OVER()

Pour chaque film, le nom du réalisateur(trice), le nom du film, le nombre de votant et le genre.
Aussi, il faut une colonne qui contient le nombre maximum de votant pour un film dont le réalisateur(trice) appartient au genre masculin ou féminin.

In [ ]:

#le nom du réalisateur, le titre original du film, le nombre de votes et le genre du réalisateur.
SELECT d.name,m.original_title, m.vote_count, d.gender,
	   MAX(m.vote_count) OVER (PARTITION BY d.gender) as max_amount_vote
#utilise une fonction de fenêtre MAX pour trouver le nombre de votes maximum pour chaque genre de réalisateur, en partitionnant la table par la colonne gender qui contient les genres. 
# La fonction de fenêtre calcule la valeur maximale de la colonne vote_count pour chaque groupe défini par le genre du réalisateur, 
# et ajoute cette valeur maximale comme colonne dans le résultat sous le nom max_amount_vote.

FROM movies m 
JOIN directors d ON d.id = m.director_id 
WHERE d.gender IS NOT 0; # ou > 0 

# Percent Rank

utiliser PERCENT_RANK afin de créer des buckets qui auront quasiment le même nombre de films.
Pour chaque film, on aimerait une information supplémentaire (donc une colonne supplémentaire). Tu vas créer plusieurs buckets:
[0 - 0.2 Percentile, 0.2 - 0.4 Percentile, 0.4 - 0.6 Percentile, 0.6 - 0.8 Percentile, 0.8 - 1 Percentile]
En plus de cette information, tu afficheras pour chaque film: le nom du réalisateur, le titre du film et le budget.


In [ ]:

# sélectionne toutes les colonnes de la requête.
SELECT *,

# crée une expression de cas qui catégorise les résultats en fonction de leur pourcentage de classement.
# Les pourcentages de classement sont divisés en cinq groupes différents, et les résultats sont étiquetés en conséquence.
CASE WHEN percent_ranking >= 0 AND percent_ranking <= 0.2 THEN '0 - 0.2 Percentile'
	   WHEN percent_ranking > 0.2 AND percent_ranking  <= 0.4 THEN '0.2 - 0.4 Percentile'
	   WHEN percent_ranking > 0.4 AND percent_ranking  <= 0.6 THEN '0.4 - 0.6 Percentile'
	   WHEN percent_ranking > 0.6 AND percent_ranking  <= 0.8 THEN '0.6 - 0.8 Percentile'
	   ELSE '0.8 - 1 Percentile'
	  END Budget_Bucket_$

# crée une sous-requête qui sélectionne les noms des réalisateurs, les titres des films qu'ils ont réalisés, le budget des films 
# et leur pourcentage de classement par rapport au budget total des films. Seuls les films ayant un budget supérieur à 0 sont inclus.
FROM (
	SELECT d.name, 
	m.original_title, 
	m.budget,
	PERCENT_RANK() OVER (ORDER BY m.budget) as Percent_ranking  # La fonction PERCENT_RANK() OVER() permet de calculer le pourcentage de classement de chaque film par rapport au budget total des films.
	FROM
	  movies m 
	      JOIN directors d ON d.id = m.director_id 
	      WHERE m.budget > 0 #pour n'inclure que les films ayant un budget supérieur à 0
	      )

# Ntile()

On aimerait avoir ici 5 Buckets, ou chacun des buckets on respectivement [752, 752, 752, 752, 751]films.
Pour chaque bucket, on veut le nombre de films, le plus petit budget de chaque bucket, le plus gros budget de chaque bucket, et enfin le budget moyen pour chaque bucket.

In [ ]:
# sélectionne les colonnes nécessaires pour les résultats finaux, qui incluent le nombre de films, le budget minimum, le budget maximum et le budget moyen

SELECT  COUNT(name) as Number_of_movies_per_bucket, 
		MIN(budget) as Minimum_budget, 
		MAX(budget) as Maximum_budget,
		ROUND(AVG(budget)) as Average_Budget # La fonction ROUND() est utilisée pour arrondir le budget moyen à un nombre entier.

#  crée une sous-requête qui sélectionne les noms des réalisateurs, les titres des films qu'ils ont réalisés, 

FROM (
	SELECT d.name,
		m.original_title,
		NTILE(5) OVER(ORDER BY m.budget) AS budget_group, # le budget des films et le groupe de budget pour chaque film, qui est déterminé en utilisant la fonction 
		budget                                            # la fonction NTILE() OVER() pour diviser les films en cinq groupes en fonction de leur budget.
	FROM movies m 
	JOIN directors d ON d.id = m.director_id 
	WHERE budget > 0  #  en filtrant les résultats pour n'inclure que les films ayant un budget supérieur à 0
	)
GROUP BY budget_group # regroupe les résultats en fonction du groupe de budget de chaque film.